In [ ]:
!pip install konlpy
!pip install gluonnlp pandas tqdm
!pip install transformers
!pip install torch

In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import time
from konlpy.tag import Kkma
import re 
import torch
from transformers import AutoModel, AutoTokenizer

tqdm.pandas() 

In [ ]:
model = AutoModel.from_pretrained("lassl/bert-ko-small")
tokenizer = AutoTokenizer.from_pretrained("lassl/bert-ko-small")

In [ ]:
stopwords=['묵', '학', '램', '맘', '쌍', 'ㄴ', '나', '필', '밖', '염', '게', '찌', '득', '전', '토', '락', '갑', '날', '롤', '싹', '천', '오', '벗', '깡', '민', '원', '훈', '본', '캡', '진', '걸', '닻', '명', '영', '뒤', '친', '덤', '이', '공', '색',
 '샷', '루', '갓', '순', '콩', '변', '빽', '무', '석', '더', '음', '랜', '먹', '롬', '수', '엠', '찬', '솔', '뭇', '쌈', '힐', '벌', '톱', 'ㅁ', '챙', '의', '님', '뷰', '세', '때', '짱', '안', '작', '렉', '윗', '정', '총', '광', '업', '데',
 '짬', '근', '야', '탄', '족', '견', '맛', '컴', '쇼', '론', '즘', '노', '립', '저', '젤', '맥', '초', '후', '란', 'ㄹ', '멍', '죽', '양', '뇌', '턴', '젖', '펜', '미', '군', '리', '콜', '요', '곳', '패', '샘', '짝', '애', '분', '담', '연', '꼴',
 '곡', '잇', '경', '엔', '못', '창', '성', '호', '밴', '랙', '글', '띠', '둑', '탓', '채', '몰', '선', '금', '굴', '비', '주', '피', '악', '투', '함', '탕', '각', '치', '맴', '태', '홍', '빗', '촌', '활', '털', '잠', '끼', '잔', '송', '술', '질', '침',
 '쟁', '단', '새', '답', '당', '댐', '직', '팔', '농', '테', '컵', '움', '볼', '물', '칠', '바', '절', '깨', '면', '돔', '핀', '병', '밥', '묘', '돌', '링', '교', '칸', '끝', '들', '처', '뜰', '실', '용', '셋', 'ㄱ', '셀', '외', '흠', '시', '런', '년',
 '최', '떼', '탐', '줄', '체', '흑', '폼', '내', '꺼', '똥', '존', '귀', '위', '웹', '대', '암', '종', '밀', '포', '강', '결', '폭', '퇴', '팩', '팬', '히', '설', '키', '콕', '조', '급', '권', '힙', '디', '윌', '소', '차', '아', '곰', '범', 'ㄷ', '현',
 '기', '킥', '널', '우', '율', '쑥', '격', '옥', '낫', '풍', '끈', '잭', '류', '톤', '월', '과', '허', '중', '논', '레', '박', '상', '열', '예', '뼈', '축', '일', '줌', '해', '쉬', '골', '청', '량', '매', '북', '증', '유', '표', '합', '판', '능', '컷',
 '쥐', '숍', '큐', '풀', '혀', '부', '던', '계', '모', '은', '국', '손', '땅', '난', '겸', '한', '망', '틈', '막', '길', '핵', '평', '잼', '울', '살', '회', '향', '감', '굿', '셈', '제', '가', '빵', '복', '첨', '핏', '동', '앤', '징', '액', '떄', '칩',
 '올', '덕', '생', '멋', '철', '속', '혈', '뜻', '베', '벽', '승', '삼', '행', '얼', '통', '자', '빔', '고', '반', '갱','콘', '즙', '신', '룰', '킹', '심', '다', '환', '힘', '습', '핫', '서', '짓', '카', '메', '역', '곁', '코', '홀', '식', '김', '검',
 '터', '녀', '알', '쌀', '효', 'ㅇ', '맨', '팝', '씨', '몫', '캔', '임', '라', '파', '장', '랩', '엘', '타', '봉', '편', '뻥', '인', '구', '탭', '짐', '딸', '훅', '간', '배', '벨', '발', '언', '왕', '펄', '충', '떡', '누', '여', '폴', '둘', 'ㅍ', '팁',
 '온', '관', '어', '폐', '건', '낙', '탑', '로', '개', '곽', '불', '삐', '녹', '씬', '하', '궁', '약', '백', '티', '닷', '황', '록', '점', '늪', '지', '보', '도', '참', '을', '탈', '붐', '겁', '마', '혼', '운', '넷', '딥', '칼', '윤', '방', '재', '러',
 '추', '만', '르', '입', '사']

In [ ]:
kkma = Kkma() 

#명사 추출
def getNVM(text):
    text=",".join(text)
    pos=""
    parsed=kkma.pos(text)
    tags=['NNP','NNG',"NP",'VA']  # 명사 형용사 추출
    for word, tag in parsed:
      if tag in tags:
        pos+=","+word
    return pos[1:]

In [ ]:
def clean_text(text):

    words=list(set(text.split(",")))
    if len(words) != 1:
      text=",".join([ word for word in words if word not in stopwords])
    elif len(words) == 1:
      text=",".join(words)
    
    return text

In [ ]:
def embedding(text):
    inputs = tokenizer.batch_encode_plus([text])
    with torch.no_grad():
      outputs = model(input_ids = torch.tensor(inputs['input_ids']))
    sentence=torch.mean(outputs[0][0], dim=0)
    return sentence

In [ ]:
tt = ['남자', '여자', '해변','로맨틱','밝음']

In [ ]:
tt = getNVM(tt)
tt

In [ ]:
tt = clean_text(tt)
tt

In [ ]:
embedding(tt)